# Problem

As is the case with point mutations, the most common type of sequencing error occurs when a single nucleotide from a read is interpreted incorrectly.

**Given**: A collection of up to 1000 reads of equal length (at most 50 bp) in FASTA format. Some of these reads were generated with a single-nucleotide error. For each read s in the dataset, one of the following applies:

1) s was correctly sequenced and appears in the dataset at least twice (possibly as a reverse complement)

2) s is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one correct read in the dataset (or its reverse complement).

**Return**: A list of all corrections in the form "[old read]->[new read]". (Each correction must be a single symbol substitution, and you may return the corrections in any order.)

In [51]:
def readTab(infile): # read in txt file
    with open(infile, 'r') as input_file:
    # read in tab-delim text
        output = []
        for input_line in input_file:
            input_line = input_line.strip()
            temp = input_line.split('\t')
            output.append(temp)
    return output
def extract_fasta(fasta):
    sequences = {}
    headers = []
    flag = ""
    for i in fasta:
        if i[0].startswith(">"):
            headers.append(i[0])
            flag = i[0]
            sequences[flag] = ""
        else:
            sequences[flag] = sequences[flag] + i[0]
    return sequences, headers
def Hamming_distance(string1,string2, normalize):
    diffs = 0
    for i in range(len(string1)):
        if string1[i] != string2[i]:
            diffs +=1
    dist = float(diffs)
    if normalize:
        dist = dist / float(len(string1))
    return float("{0:.5f}".format(dist))
def reverse_complement(sequence):
    complement = {"A":"T","C":"G","G":"C","T":"A"}
    rev = ""
    for i in sequence[::-1]:
        rev += complement[i] 
    return rev
def occurences_corpus(sequences,ID):
    corpus = sequences.values()
    cnt=0
    for i in corpus:
        if sequences[ID] == i:
            cnt+=1
        if reverse_complement(sequences[ID]) == i:
            cnt+=1
    return cnt
def detect_error_reads(sequences,headers):
    # Go here for help http://rosalind.info/problems/corr/questions/#comment-11608
    errors = []
    for i in headers:
        if occurences_corpus(sequences,i) == 1:
            for j in headers:
                if int(Hamming_distance(sequences[i],sequences[j],False)) == 1: # Hamming distance criteria
                    if (j,i) not in errors:
                        errors.append((i,j))
                elif int(Hamming_distance(sequences[i],reverse_complement(sequences[j]),False)) == 1: # check rev comp
                    if (j,i) not in errors:
                        errors.append((i,j))
                else:
                    pass
    return errors
def correct_error_reads(sequences,errors):
    lines = []
    for i in errors:
        if int(Hamming_distance(sequences[i[0]],sequences[i[1]],False))==1:
            temp = sequences[i[0]] + "->" + sequences[i[1]]
            if temp not in lines:
                lines.append(temp)
        if int(Hamming_distance(reverse_complement(sequences[i[0]]),sequences[i[1]],False))==1:
            temp = sequences[i[0]] + "->" + reverse_complement(sequences[i[1]])
            if temp not in lines:
                lines.append(temp)
    return "\n".join(lines)

In [52]:
test_seq,test_head = extract_fasta(readTab("ErrorsInReads.fasta"))
answer = """TTCAT->TTGAT
GAGGA->GATGA
TTTCC->TTTCA"""

In [54]:
errors = detect_error_reads(test_seq,test_head)
output = correct_error_reads2(test_seq,errors)
#print errors
print output

True


In [56]:
test_seq2,test_head2 = extract_fasta(readTab("rosalind_corr.txt"))
errors = correct_error_reads(test_seq2,detect_error_reads2(test_seq2,test_head2))
print errors

...
